# Elastic Search
# 4. 데이터 검색
## 4.1 검색 API
## URI vs Request Body

- URI <br>
POST movie_search/_search?q=movieNmEn: * AND prtdYear:2017&analyze_wildcard=true&from=0&size=5&sort=_score:desc,movieCd:asc&_source_include=movieCd,movieNm,movieNmEn,typeNm

- Request Body <br>
POST movie_search/_search
{
  "query": {
    "query_string": {
      "default_field": "movieNmEn",
      "query": "Family"
    }
  }
}
<br>
POST movie_search/_search
{
  "query": {
    "query_string": {
      "default_field": "movieNmEn",
      "query": "movieNmEn: * OR prdtYear:2017"
    }
  },
  "from": 0,
  "size": 5,
  "sort": [
    {
      "_score": {
        "order": "desc"
      },
      "movieCd": {
        "order": "asc"
      }
    }
  ],
  "_source": [
    "movieCd", "movieNm", "movieNmEn", "typeNm"
    ]
}

## 4.2 Query DSL 이해하기
### 4.2.3 Query DSL의 주요 파라미터
- Multi Index 검색 <br>
POST movie_search,movie_auto/_search
{
  "query" : {
    "term": {
      "repGenreNm": "다큐멘터리"
    }
  }
}

- 쿼리 결과 페이징 : 검색결과 페이지별로 나타낼 때 사용
<!-- 첫번 째 페이지 요청 -->
POST movie_search/_search
{
  "from": 0,
  "size": 5,
  "query": {
    "term": {
        "repNationNm": "한국"
    }
  }
}
<br>
<!-- 두번 째 페이지 요청 -->
POST movie_search/_search
{
  "from": 5,
  "size": 5,
  "query": {
    "term": {
        "repNationNm": "한국"
    }
  }
}

- 쿼리 결과 정렬 <br>
POST movie_search/_search
{
  "query": {
    "term": {
        "repNationNm": "한국"
    }
  },
  "sort": [
    {
      "prdtYear" : {
        "order": "asc"
      }
      "_score": {
        "order": "desc"
      }
    }
  ]
}

- _source 필드 필터링 <br>
POST movie_search/_search
{
  "query": {
    "term": {
        "repNationNm": "한국"
    }
  },
  "_source": "movieNm"
}

- 범위 검색 <br>
POST movie_search/_search
{
  "query": {
    "range": {
      "prdtYear": {
          "gte": "2016",
          "lte": "2017"
      }
    }
  }
}

- operator 설정 : default "OR"이기에 쿼리 띄어쓰기 없이 검색하기 위해 "AND" 사용 <br>
POST movie_search/_search
{
  "query": {
    "match": {
      "movieNm": {
          "query": "자전차왕 엄복동",
          "operator": "and"
      }
    }
  }
}

- minimum_should_match 설정 : 결과에 포함할 텀의 최소 개수 지정 (AND 효과) <br>
POST movie_search/_search
{
  "query": {
    "match": {
      "movieNm": {
          "query": "자전차왕 엄복동",
          "minimum_should_match": 2
      }
    }
  }
}

- fuzziness 설정 : 레벤슈타인 편집 거리 알고리즘 기반으로 문서간의 유사도 측정 (거리값), 한국어에는 적용 어려움 <br>
POST movie_search/_search
{
  "query": {
    "match": {
      "movieNm": {
          "query": "Fli High",
          "fuzziness": 1
      }
    }
  }
}

- boost 설정 : 관련성이 높은 필드나 키워드에 가중치를 더 줄 수 있음 <br>
POST movie_search/_search
{
  "query": {
    "multi_match": {
        "query": "Fly",
        "fields": ["movieNm^3", "movieNmEn"]
    }
  }
}

## 4.3 Query DSL의 주요 쿼리
### 4.3.1 Match All Query
POST movie_search/_search
{
  "query": {
    "match_all": {}
  }
}

### 4.3.2 Match Query
POST movie_search/_search
{
  "query": {
    "match": {
        "movieNm": "그대 장미"
    }
  }
}

### 4.3.3 Multi Match Query : fields에서 해당 하는 쿼리 있는 경우 반환
POST movie_search/_search
{
  "query": {
    "multi_match": {
        "query": "가족",
        "fields": ["movieNm", "movieNmEn"]
    }
  }
}

### 4.3.4 Term Query : Keyword 데이터 타입을 대상으로 하며, 일반적으로 숫자, Keyword, 날짜 데이터 쿼리에 사용함 / 영문의 경우에는 대소문자가 다를 경우 검색되지 않으므로 주의!
POST movie_search/_search
{
  "query": {
    "term": {
        "genreAlt": "코미디"
    }
  }
}

### 4.3.5 Bool Query
### 4.3.5 Bool Query
POST movie_search/_search
{
  "query": {
    "bool": {
      "must": [
          {
            "term": {
                "repGenreNm": "코미디"
            }
          },
          {
            "match" : {
                "repNationNm": "한국"
            }
          }
      ],
      "must_not": [
        {
          "match": {
            "typeNm": "단편"
          }
        }
      ]
    }
  }
}

### 4.3.6 Query String
POST movie_search/_search
{
  "query": {
    "query_string": {
        "default_field": "movieNm",
        "query" : "(가정) AND (어린이 날)"
    }
  }
}

### 4.3.7 Prefix Query
POST movie_search/_search
{
  "query": {
    "prefix": {
        "movieNm" : "자전차"
    }
  }
}

### 4.3.8 Exists Query : Null값을 제외한 데이터를 리턴
POST movie_search/_search
{
  "query": {
    "exists": {
        "field": "movieNm"
    }
  }
}

### 4.3.9 Wildcard Query
- "?" : 지정된 위치의 한글자가 다른 경우의 문서를 찾음
- "*" : 문자의 길이와 상관 없이 와일드카드와 일치하는 모든 문서 찾음
<br>

POST movie_search/_search
{
  "query": {
    "wildcard": {
        "typeNm": "장?"
    }
  }
}

### 4.3.10 Nested Query : SQL의 Join과 유사한 기능을 제공
PUT movie_nested
{
  "settings": {
    "number_of_shards": 5,
    "number_of_replicas": 1
  },
  "mappings": {
    "_doc": {
      "properties": {
        "repGenreNm": {
          "type": "keyword"
        },
        "companies": {
          "type": "nested",
          "properties": {
            "companyCd" : {
              "type": "keyword"
            },
            "companyNm": {
              "type" : "keyword"
            }
          }
        }
      }
    }
  }
}
<br>
GET movie_nested/_doc/1
{
  "movieCd": "20184623",
  "movieNm": "바람난 아내들2",
  "movieNmEn": "",
  "prdtYear": "2018",
  "openDt": "",
  "typeNm": "장편",
  "prdtStatNm": "개봉예정",
  "nationAlt": "한국",
  "genreAlt": "멜로/로맨스",
  "repNationNm": "한국",
  "repGenreNm": "멜로/로맨스",
  "companies": [
    {
      "companyCd": "20173401",
      "companyNm": "(주)케이피에이기획"
    }
  ]
}
<br>
GET movie_nested/_search
{
  "query": {
    "bool": {
      "must": [
        {
          "term": {
              "repGenreNm": "멜로/로맨스"
          }
        },
        {
          "nested": {
            "path": "companies",
            "query": {
              "bool": {
                "must": [
                  {
                    "term": {
                        "companies.companyCd": "20173401"
                    }
                  }
                ]
              }
            }
          }
        }
      ]
    }
  }
}

## 4.4 부가적인 검색 API
### 4.4.1 효율적인 검색을 위한 환경설정 : 
- 동적 분배 방식으로 요청하는 설정 <br>
PUT _cluster/settings
{
  "transient": {
    "cluster.routing.use_adaptive_replica_selection": true
  }
}
- 글로벌 타임아웃 설정 <br>
PUT _cluster/settings
{
  "transient": {
    "search.default_search_timeout": "1s"
  }
}

### 4.4.2 Search Shards API : 검색이 수행되는 노드 및 샤드에 대한 정보 확인 가능
POST movie_search/_search_shards

### 4.4.3 Multi Search API : 여러 건의 검색 요청을 통합해서 한 번에 요청하고 한 번에 결과를 종합해서 받음
POST _msearch
{"index": "movie_search"}
{"query": {"match_all": {}}, "from": 0, "size": 10}
{"index": "movie_search"}
{"query": {"match_all": {}}, "from": 0, "size": 10}

### 4.4.4 Count API : 내용 출력 없이 카운트만 진행
- URI 검색도 가능
POST movie_search/_count
{
  "query": {
    "query_string": {
      "default_field": "prdtYear",
      "query": "2017"
    }
  }
}

### 4.4.5 Validate API : valid 값이 true/false로 출력 (쿼리 오류시, false return)
- URI 검색도 가능
POST movie_search/_validate/query
{
  "query": {
    "match": {
      "prdtYear": 2017
    }
  }
}


### 4.4.6 Explain API : _score에 대해서 계산하는 파라미터 등을 확인 가능
POST movie_search/_doc/ejzJqmkBjjM-ebDb8PsR/_explain
{
  "query": {
    "term": {
      "prdtYear": 2017
    }
  }
}

### 4.4.7 Profile API : 쿼리에 대한 내용을 상세하게 설명 (결과가 방대함 / 각 샤드별로 얼마나 많은 시간 소요시간 파악)
POST movie_search/_search
{
  "profile": true, 
  "query": {
    "match_all": {}
  }
}

# 5. 데이터 집계
## 5.1 집계
### 5.1.3 실습 데이터 살펴보기
GET /apache-web-log/_search?size=0
{
  "query": {
    "match_all": {}
  },
  "aggs": {
    "region_count": {
      "terms": {
        "field": "geoip.city_name.keyword",
        "size": 20
      }
    }
  }
}

### 5.1.4 Aggregation API
- 버킷 집계 / 메트릭 집계 / 파이프라인 집계 / 행렬 집계
- 엘라스틱서치는 집계를 중첩해 활용 가능 but 중첩할수록 성능은 하락
- 진행 방식: query를 통한 값 반환 -> 쿼리를 통한 값들을 집계 수행

## 5.2 메트릭 집계
### 5.2.1 합산 집계
- divide_value를 통해서 MB로 환산
GET /apache-web-log/_search?size=0
{
  "query": {
    "constant_score": {
      "filter": {
        "match": {"geoip.city_name": "Paris"}
      }
    }
  }, 
  "aggs": {
    "total_bytes": {
      "sum": {
        "script": {
          "lang": "painless",
          "source": "doc.bytes.value/params.divide_value",
          "params": {
            "divide_value": 1000
          }
        }
      }
    }
  }
}

### 5.2.2 평균 집계
GET /apache-web-log/_search?size=0
{
  "query": {
    "constant_score": {
      "filter": {
        "match": {"geoip.city_name": "Paris"}
      }
    }
  }, 
  "aggs": {
    "avg_bytes": {
      "avg": {
        "field": "bytes"
      }
    }
  }
}

### 5.2.3 최솟값 집계
GET /apache-web-log/_search?size=0
{
  "query": {
    "constant_score": {
      "filter": {
        "match": {"geoip.city_name": "Paris"}
      }
    }
  }, 
  "aggs": {
    "min_bytes": {
      "min": {
        "field": "bytes"
      }
    }
  }
}

### 5.2.4 최대값 집계
GET /apache-web-log/_search?size=0
{
  "query": {
    "constant_score": {
      "filter": {
        "match": {"geoip.city_name": "Moscow"}
      }
    }
  }, 
  "aggs": {
    "max_bytes": {
      "max": {
        "field": "bytes"
      }
    }
  }
}

### 5.2.5 개수 집계
GET /apache-web-log/_search?size=0
{
  "query": {
    "constant_score": {
      "filter": {
        "match": {"geoip.city_name": "Moscow"}
      }
    }
  }, 
  "aggs": {
    "bytes_count": {
      "value_count": {
        "field": "bytes"
      }
    }
  }
}

### 5.2.6 통계 집계
GET /apache-web-log/_search?size=0
{
  "query": {
    "constant_score": {
      "filter": {
        "match": {"geoip.city_name": "Moscow"}
      }
    }
  }, 
  "aggs": {
    "bytes_stats": {
      "stats": {
        "field": "bytes"
      }
    }
  }
}

### 5.2.7 확장 통계 집계
GET /apache-web-log/_search?size=0
{
  "query": {
    "constant_score": {
      "filter": {
        "match": {"geoip.city_name": "Moscow"}
      }
    }
  }, 
  "aggs": {
    "bytes_extended_stats": {
      "extended_stats": {
        "field": "bytes"
      }
    }
  }
}

### 5.2.8 카디널리티 집계 : 중복된 값을 제외한 고유한 값에 대한 집계를 수행
- 미국 각 지역에서 데이터 유입 확인 <br>
GET /apache-web-log/_search?size=0
{
  "query": {
    "constant_score": {
      "filter": {
        "match": {"geoip.country_name": "United States"}
      }
    }
  }, 
  "aggs": {
    "us_city_names": {
      "terms": {
        "field": "geoip.city_name.keyword"
      }
    }
  }
}
<br>
- 동일한 필드 값에 대해서도 집계 연산 수행되지 않기 위해 카디널리티 집계 이용 <br>
GET /apache-web-log/_search?size=0
{
  "query": {
    "constant_score": {
      "filter": {
        "match": {"geoip.country_name": "United States"}
      }
    }
  }, 
  "aggs": {
    "us_cardinality": {
      "cardinality": {
        "field": "geoip.city_name.keyword"
      }
    }
  }
}

### 5.2.9 백분위 수 집계 : 유입된 데이터 크기 분포 확인
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "bytes_percentiles": {
      "percentiles": {
        "field": "bytes",
        "percents": [10, 20, 30, 40, 50, 60, 70, 80, 90]
      }
    }
  }
}

### 5.2.10 백분위 수 랭크 집계 : 특정 필드 수치를 통해 백분위의 어느 구간에 속하는지 확인
GET /apache-web-log/_search?size=0
{
  "aggs": {
    "bytes_percentile_ranks": {
      "percentile_ranks": {
        "field": "bytes",
        "values": [5000, 10000]
      }
    }
  }
}

### 5.2.11 지형 경계 집계
- close <br>
POST apache-web-log/_close
- 스냅숏 복원 <br>
curl -XPOST "http://localhost:9200/_snapshot/apache-web-log/applied-mapping/_restore" -u elastic:bigdata
- 복원 확인 <br>
GET /_cat/indices/apache*?v&pretty
- 매핑정보 확인 <br>
GET /apache-web-log-applied-mapping/_mapping/field/geoip.location
- 집계 수행 <br>
GET /apache-web-log-applied-mapping/_search?size=0
<br>
GET /apache-web-log-applied-mapping/_search?size=0
{
  "query": {
    "constant_score": {
      "filter": {
        "match": {
          "geoip.continent_code" : "EU"
        }
      }
    }
  }
}

### 5.2.12 지형 중심 집계
GET /apache-web-log-applied-mapping/_search?size=0
{
  "aggs": {
    "centroid": {
      "geo_centroid": {
        "field": "geoip.location"
      }
    }
  }
}